In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import random
import requests  # To download the text dataset

# 1. Load and preprocess the text data
url = "https://raw.githubusercontent.com/brunoklein99/deep-learning-notes/master/shakespeare.txt"
response = requests.get(url)
text = response.text.lower()  # Convert to lowercase for consistency

print(f"Length of text: {len(text)} characters")
print(text[:500])  # Print first 500 characters to inspect

# 2. Create character-level mappings
chars = sorted(list(set(text)))
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

print(f"Total characters: {len(text)}")
print(f"Unique characters: {len(chars)}")

# 3. Prepare training sequences
maxlen = 40  # Length of input sequences
step = 3     # Step size between sequences
sentences = []  # Input sequences
next_chars = []  # Output characters (labels)

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])

print(f"Number of sequences: {len(sentences)}")

# Vectorize the data
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# 4. Build the LSTM model
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(chars))),
    Dense(len(chars), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.01),
             loss='categorical_crossentropy')

model.summary()

# 5. Train the model
history = model.fit(X, y, batch_size=128, epochs=30, validation_split=0.2)

# 6. Text generation function with temperature
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(seed_text, length=400, temperature=1.0):
    generated = seed_text
    print(f'----- Generating with temperature {temperature}')
    print(f'----- Seed: "{seed_text}"')

    for i in range(length):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated[-maxlen:]):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        generated += next_char

    print(generated)
    print()
    return generated

# 7. Generate text with different temperatures
seed_text = "shall i compare thee to a summer's day?\n"
print("----- Original text excerpt -----")
print(text[1000:1400])
print("\n----- Generated text -----")

# Generate with different temperature settings
generate_text(seed_text, temperature=0.2)
generate_text(seed_text, temperature=0.5)
generate_text(seed_text, temperature=1.0)
generate_text(seed_text, temperature=1.5)

Length of text: 94275 characters
the sonnets

by william shakespeare

from fairest creatures we desire increase,
that thereby beauty's rose might never die,
but as the riper should by time decease,
his tender heir might bear his memory:
but thou contracted to thine own bright eyes,
feed'st thy light's flame with self-substantial fuel,
making a famine where abundance lies,
thy self thy foe, to thy sweet self too cruel:
thou that art now the world's fresh ornament,
and only herald to the gaudy spring,
within thine own bud buriest
Total characters: 94275
Unique characters: 38
Number of sequences: 31412


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 128)                 │          85,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 38)                  │           4,902 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 90,406 (353.15 KB)

 Trainable params: 90,406 (353.15 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 29s 135ms/step - loss: 2.8148 - val_loss: 2.1542
Epoch 2/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 38s 120ms/step - loss: 2.0644 - val_loss: 1.9684
Epoch 3/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 24s 122ms/step - loss: 1.8358 - val_loss: 1.8777
Epoch 4/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 41s 120ms/step - loss: 1.6937 - val_loss: 1.8235
Epoch 5/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 40s 114ms/step - loss: 1.5833 - val_loss: 1.8242
Epoch 6/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 42s 120ms/step - loss: 1.4834 - val_loss: 1.8279
Epoch 7/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 26s 130ms/step - loss: 1.3936 - val_loss: 1.8343
Epoch 8/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 40s 123ms/step - loss: 1.3021 - val_loss: 1.8666
Epoch 9/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 40s 120ms/step - loss: 1.2366 - val_loss: 1.8932
Epoch 10/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 23s 118ms/step - loss: 1.2054 - val_loss: 1.9113
Epoch 11/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 40s 114ms/step - loss: 1.1446 - val_loss: 1.9522
Epoch 12/30
197/197

"shall i compare thee to a summer's day?\nmoke hand no ell, you, thougaim'x joought's bud.'s desined,\nand than war whay i'rovid, therebicdard\nweaps of a sun debast be oldpeds i nexreserdect:\nthat 'gainst thee are bran wrecker agell link,\nof beauty excubs thy soightless thenst excel.o hand,  \nthis stoulling me heatt) load, i veals a confeni de,\nalone thing? ressensee by trivil lewfon:'s brood,\nlif in are dreds do charthing as p;sul my:\nbur"